In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
layer = nn.Linear(10,10)
x1 = torch.randn(16, 10)
x2 = torch.randn(16, 10)
y1 = F.normalize(layer(x1))
y2 = F.normalize(layer(x2))

loss1 = F.mse_loss(y1, y2, reduction='none').sum(dim=-1).mean()
loss2 = (2 - 2 * (y1 * y2).sum(dim=-1)).mean()

loss1.backward(retain_graph=True)
grad1 = layer.weight.grad.clone()
layer.weight.grad.zero_()
loss2.backward()
grad2 = layer.weight.grad.clone()

assert torch.allclose(grad1, grad2)
